# このノートブックは？
* CreateInitialData.ipynbで作成したデータを利用し、特徴量作成、LightGBMでの学習、予測結果のファイル出力を行う
* NASDAQを対象に学習し、予測結果のうちAMEX分のみをこの後に利用

## Google Driveのマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/'My Drive'/'PROBSPACE'/'StockPricePrediction'

Mounted at /content/drive
/content/drive/My Drive/PROBSPACE/StockPricePrediction


## ライブラリのインストール、インポート

In [ ]:
import src.install_libraries
import category_encoders as ce
from src.common_functions import *
from src.create_features import *
import lightgbm as lgb

  Created wheel for ta: filename=ta-0.8.0-py3-none-any.whl size=28895 sha256=0eeda9bf609364c0d1d600aaeb11cdf14313fe7d56a910d17a15df01100ae4ac
  Stored in directory: /root/.cache/pip/wheels/7e/da/86/65cba22446ae2ef148de2079907264ef27feecfb7f51a45e0d
Successfully built ta


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## データの読み込み

In [ ]:
price_df2, company_df2, submission_df = read_initial_data()

In [ ]:
# 乱数固定
SEED = 42
seed_everything(SEED)

set_pandas_options()

## 特徴量追加

In [ ]:
def create_features(df, group_col, val_col1, val_col2, prefix):
  res = pd.DataFrame(index=df.index)
  group_df = df.groupby(group_col) if group_col else df.copy()

  # ラグ特徴量
  with Timer(f'{prefix}ラグ特徴量'):
    lag_df = create_lags(group_df, val_col1, [1, 2, 3, 4], prefix)
    res = res.join(lag_df)

  # 単純移動平均、標準偏差
  with Timer(f'{prefix}単純移動平均'):
    roll_df = create_rolls(group_df, val_col1, [1], [52], prefix)
    res = res.join(roll_df)

  # # EMA GAP
  # with Timer(f'{prefix}EMA_GAP'):
  #   ema_df = create_ema_gap(df, group_df, group_col, val_col1, [13, 26], prefix)
  #   res = res.join(ema_df)

  # # Return
  # with Timer(f'{prefix}Return'):
  #   return_df = create_return(group_df, val_col2, [13, 26], prefix)
  #   res = res.join(return_df)

  # # HV
  # with Timer(f'{prefix}HV'):
  #   hv_df = create_hv(df, group_df, group_col, val_col2, [13, 26], prefix)
  #   res = res.join(hv_df)

  # # Skew
  # with Timer(f'{prefix}Skew'):
  #   skew_df = create_skew(df, group_df, group_col, val_col2, [13, 26], prefix)
  #   res = res.join(skew_df)

  # # Kurt
  # with Timer(f'{prefix}Kurt'):
  #   kurt_df = create_kurt(group_df, val_col2, [13, 26], prefix)
  #   res = res.join(kurt_df)

  # # Var
  # with Timer(f'{prefix}Var'):
  #   var_df = create_var(group_df, val_col1, [13, 26], prefix)
  #   res = res.join(var_df)

  # # MIN, MAX
  # if group_col:
  #   with Timer(f'{prefix}MIN, MAX'):
  #     min_max_df = create_min_max(group_df, val_col1, [13, 26], prefix)
  #     res = res.join(min_max_df)

  # # Up, Down
  # if group_col:
  #   with Timer(f'{prefix}Up, Down'):
  #     up_down_df = create_up_down(df, group_col, val_col1, [13, 26], prefix)
  #     res = res.join(up_down_df)

  # # ATR
  # if group_col:
  #   with Timer(f'{prefix}ATR'):
  #     atr_df = create_atr(group_df, val_col2, prefix)
  #     res = res.join(atr_df)

  # # RSI
  # if group_col:
  #   with Timer(f'{prefix}RSI'):
  #     rsi_df = create_rsi(group_df, val_col2, prefix)
  #     res = res.join(rsi_df)

  # # MACD
  # if group_col:
  #   with Timer(f'{prefix}MACD'):
  #     macd_df = create_macd(group_df, val_col2, prefix)
  #     res = res.join(macd_df)

  # # BB
  # if group_col:
  #   with Timer(f'{prefix}BB'):
  #     bb_df = create_bb(group_df, val_col2, prefix)
  #     res = res.join(bb_df)

  # # Stochastic Oscillator
  # if group_col:
  #   with Timer(f'{prefix}Stochastic Oscillator'):
  #     stock_df = create_stock(group_df, val_col2, prefix)
  #     res = res.join(stock_df)

  # # Parabolic Stop and Reverse
  # if group_col:
  #   with Timer(f'{prefix}Parabolic Stop and Reverse'):
  #     parabolic_df = create_parabolic(group_df, val_col2, prefix)
  #     res = res.join(parabolic_df)

  # # KST Oscillator
  # if group_col:
  #   with Timer(f'{prefix}KST Oscillator'):
  #     kst_df = create_kst(group_df, val_col2, prefix)
  #     res = res.join(kst_df)

  return res


def add_feature(df):
  res = df.copy()

  # 日付
  res['Year'] = res.Date.dt.year
  res['Month'] = res.Date.dt.month
  res['Day'] = res.Date.dt.day
  res['WeekOfYear'] = res.Date.dt.isocalendar().week.astype(int)

  # ターゲットエンコーディング
  target_columns = ['Sector', 'Industry', 'List1', 'List2']
  ms = ce.MEstimateEncoder(cols=target_columns, m=1.0)
  ms_df = ms.fit_transform(res[target_columns], res.y_diff_norm)
  res['List'] = res.List1
  res.loc[:, target_columns] = ms_df.loc[:, target_columns]

  # 各種特徴量作成
  res = res.join(create_features(res, 'id', 'y_diff_norm', 'y_prev', ''))

  # 市場平均価格を求める
  prefix = 'Market_'
  tmp_market_df = res[['Date']].copy()
  tmp_market_df['y'] = np.expm1(res.y)
  market_df = tmp_market_df.groupby('Date').mean().apply(np.log1p).reset_index()
  market_df['y_prev'] = market_df['y'].transform(lambda x: x.shift(1).fillna(method='bfill'))
  market_df['y_diff'] = market_df['y'] - market_df['y_prev']
  market_df['y_diff_std'] = market_df['y'].std()
  market_df['y_diff_norm'] = market_df['y_diff'] / market_df['y_diff_std']

  market_columns = [prefix + c if c != 'Date' else c for c in market_df.columns]
  market_df.columns = market_columns

  # 市場平均の各種特徴量作成、結合
  market_df = market_df.join(create_features(market_df, None, 'Market_y_diff_norm', 'Market_y_prev', prefix))
  res = res.merge(market_df, on='Date')
  market_columns = market_df.drop('Date', axis=1).columns.values.tolist()

  # 各銘柄の特徴量を相対化
  # シャープレシオ
  # with Timer(f'Sharp Ratio'):
  #   group_df1 = res.groupby('id')
  #   group_df2 = market_df
  #   sharp_df = sharp_ratio(group_df1, group_df2, 'y_prev', [13, 26])
  #   res = res.join(sharp_df)

  # 対象列の設定
  replace_columns = []
  # replace_columns += [f'lag_{i}' for i in [1, 2, 3, 4]]
  # replace_columns += [f'rmean_{i}_{j}' for i in [1] for j in [52]]
  # replace_columns += [f'rstd_{i}_{j}' for i in [1] for j in [52]]
  # replace_columns += [f'EMA_GAP_{i}' for i in [13, 26]]
  # replace_columns += [f'Return_{i}' for i in [13, 26]]
  # replace_columns += [f'HV_{i}' for i in [13, 26]]
  # replace_columns += [f'Skew_{i}' for i in [13, 26]]
  # replace_columns += [f'Kurt_{i}' for i in [13, 26]]
  # replace_columns += [f'Var_{i}' for i in [13, 26]]

  # 市場平均価格の特徴量に対する各銘柄の特徴量を作成
  replace_columns_market = [prefix + c for c in replace_columns]
  feature_from_market = pd.DataFrame()
  for c1, c2 in zip(replace_columns, replace_columns_market):
    feature_from_market[f'{c1}_from_Market'] = res[c1] - res[c2]
  res = res.drop(replace_columns+market_columns, axis=1).join(feature_from_market).join(res[market_columns])

  # 不要な特徴量の削除
  drop_columns = []
  drop_columns += market_columns
  res.drop(drop_columns, axis=1, inplace=True, errors='ignore')

  # # PCA
  # with Timer('PCA'):
  #   res = res.join(pca(res, 2))

  # # K-Means
  # with Timer('K-Means'):
  #   res = res.join(k_means(res, 3))

  return res


price_df3 = add_feature(price_df2)
price_df3

2021-12-13 13:49:35 ラグ特徴量 実行時間: 0.15分
2021-12-13 13:49:40 単純移動平均 実行時間: 0.08分
2021-12-13 13:49:40 Market_ラグ特徴量 実行時間: 0.00分
2021-12-13 13:49:40 Market_単純移動平均 実行時間: 0.00分


,Date,id,y,y_prev,y_diff,y_diff_std,y_diff_norm,IPOyear,Sector,Industry,List1,List2,Year,Month,Day,WeekOfYear,List,lag_1,lag_2,lag_3,lag_4,rmean_1_52,rstd_1_52
0,2011-11-13,VGSH,4.042036,4.042036,0.0,0.017516,0.0,NaN,0.007648,0.007648,0.003676,0.003665,2011,11,13,45,NASDAQ,NaN,NaN,NaN,NaN,NaN,NaN
1,2011-11-13,JEF,2.925793,2.925793,0.0,0.141216,0.0,NaN,0.006327,0.003095,0.005265,0.005255,2011,11,13,45,NYSE,NaN,NaN,NaN,NaN,NaN,NaN
2,2011-11-13,IVZ,2.692657,2.692657,0.0,0.215237,0.0,NaN,0.006327,0.005636,0.005265,0.005255,2011,11,13,45,NYSE,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-11-13,KTCC,1.752672,1.752672,0.0,0.207799,0.0,1983.0,0.003404,0.003106,0.003676,0.003665,2011,11,13,45,NASDAQ,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-11-13,FBZ,2.695899,2.695899,0.0,0.224060,0.0,NaN,0.007648,0.007648,0.003676,0.003665,2011,11,13,45,NASDAQ,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376755,2019-11-24,TYG,NaN,2.884177,NaN,0.141783,NaN,2004.0,0.007648,0.007648,0.005265,0.005255,2019,11,24,47,NYSE,-0.087223,-0.051072,-0.207735,0.031837,-0.018841,0.221444
1376756,2019-11-24,VIRC,NaN,1.543298,NaN,0.264580,NaN,NaN,0.005102,0.003960,0.003676,0.003665,2019,11,24,47,NASDAQ,-0.196693,-0.120716,0.059876,0.037910,-0.010175,0.153233
1376757,2019-11-24,BIS,NaN,2.754241,NaN,1.087817,NaN,NaN,0.007648,0.007648,0.003676,0.003665,2019,11,24,47,NASDAQ,-0.059202,-0.017845,-0.009060,-0.071397,-0.005512,0.064397
1376758,2019-11-24,WOOD,NaN,4.184112,NaN,0.225542,NaN,NaN,0.007648,0.007648,0.003676,0.003665,2019,11,24,47,NASDAQ,-0.047620,0.020729,0.060736,0.044388,0.004269,0.127733


## 学習、予測

In [ ]:
lgb_params = {
  'objective': 'regression',
  'importance_type': 'gain',
  'metric': 'rmse',
  'seed': SEED,
  'n_jobs': -1,
  'verbose': -1,
  'n_estimators': 500, 
  'boosting_type': 'gbdt',
  'colsample_bytree': 0.559423814132516,
  'learning_rate': 0.08009483103711426,
  'min_child_samples': 251,
  'num_leaves': 197,
  'subsample': 0.8201736103298911,
  'subsample_freq': 9,
  'max_bin': 159,
}

other_params = {
  'early_stopping_rounds': 100,
}

In [ ]:
def train_evaluate(price_df, submission_df):
  test_size = 0.01

  # データの分離
  train_df = price_df.loc[price_df.Date < Config.submission_date].reset_index(drop=True)
  submit_df = price_df.loc[price_df.Date == Config.submission_date].reset_index(drop=True)
  original_train_df = train_df.copy()

  if test_size == 0:
    train_df = train_df.loc[train_df.List=='NASDAQ', :]
  else:
    unique_id = train_df.loc[train_df.List=='NASDAQ', :].id.unique()
    train_id, test_id = train_test_split(unique_id, test_size=test_size, random_state=SEED)
    train_df = train_df.loc[train_df.id.isin(train_id), :]

  useless_cols = [
    'y', 'y_prev', 'y_diff', 'y_diff_std', 'y_diff_norm', 'Date', 'id',
    'Market_y', 'Market_y_prev', 'Market_y_diff', 'Market_y_diff_std', 'Market_y_diff_norm', 'List'
  ]
  usable_cols = train_df.columns[~train_df.columns.isin(useless_cols)]
  target_col = 'y_diff_norm'

  x_train = train_df[usable_cols]
  y_train = train_df[target_col]
  x_submit = submit_df[usable_cols]

  # 学習
  y_diff_std = train_df['y_diff_std']
  y_submit = []

  unique_id = train_df.id.unique()
  kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
  for fold, (tr_group_idx, va_group_idx) in enumerate(kf.split(unique_id)):
    tr_groups, va_groups = unique_id[tr_group_idx], unique_id[va_group_idx]
    tr_idx = train_df.id.isin(tr_groups)
    vl_idx = train_df.id.isin(va_groups)

    x_tr_fold = x_train[tr_idx]
    y_tr_fold = y_train[tr_idx]
    x_vl_fold = x_train[vl_idx]
    y_vl_fold = y_train[vl_idx]

    model = lgb.LGBMRegressor(**lgb_params)
    model.fit(
        x_tr_fold, y_tr_fold,
        eval_set=(x_vl_fold, y_vl_fold),
        eval_metric='rmse',
        verbose=False,
        early_stopping_rounds=other_params['early_stopping_rounds'],
    )

    y_submit.append(model.predict(x_submit))

  # submissionデータ作成
  submission_df['y'] = submit_df['y']
  submission_df['y_prev'] = submit_df['y_prev']
  submission_df['y_diff_std'] = submit_df['y_diff_std']
  submission_df['y_diff_norm'] = submit_df['y_diff_norm']
  submission_df['pred_y'] = np.mean(y_submit, axis=0) * submit_df['y_diff_std'].values + submit_df['y_prev'].values
  submission_df['exp_pred_y'] = np.expm1(submission_df['pred_y'])
  submission_df.loc[submission_df['exp_pred_y'] < 0, 'exp_pred_y'] = 0
  submission_df['pred_y_diff_norm'] = np.mean(y_submit, axis=0)
  submission_df['List'] = submit_df['List']
  submission_df['List1'] = submit_df['List1']

  return model, submission_df


model, submission_df = train_evaluate(price_df3, submission_df)

## submissionファイル作成

In [ ]:
submission_df2 = submission_df[['id', 'exp_pred_y']]
submission_df2.columns = ['id', 'y']

In [ ]:
submission_df2.to_csv(f'{Config.intermediate_dir_name}/submission_lgb_AMEX.csv', index=False)